<a href="https://colab.research.google.com/github/ovimihai/MaxMind-python-mmdb-to-csv-converter/blob/main/mmdb_converter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MaxMind mmdb to csv converter
Can run directly in Google Colab - no VMs required

## Steps
1. Open this notebook in Google Colab
2. upload your mmdb to your Google Drive
3. start the conversion

In [ ]:
!pip install maxminddb

### Get from Google Drive
- Connect your Google Drive to Colab
- Upload your mmdb file

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/MyDrive/" + "path/"

### Or download directly here

In [ ]:
# !wget https://github.com/maxmind/MaxMind-DB/raw/main/test-data/GeoIP2-City-Test.mmdb

In [ ]:
# path = "/content/"

In [ ]:
import maxminddb
import copy
import pandas as pd

In [ ]:
from ipaddress import IPv4Network, IPv6Network


def iterable(self):
    
    if self._metadata.ip_version == 4:
        start_node = self._start_node(32)
        start_network = IPv4Network((0, 0))
    else:
        start_node = self._start_node(128)
        start_network = IPv6Network((0, 0))

    search_nodes = [(start_node, start_network)]
    while search_nodes:
        node, network = search_nodes.pop()

        if network.version == 6:
            naddr = network.network_address
            if naddr.ipv4_mapped or naddr.sixtofour:
                # skip IPv4-Mapped IPv6 and 6to4 mapped addresses, as these are
                # already included in the IPv4 part of the tree below
                continue
            elif int(naddr) < 2 ** 32 and network.prefixlen == 96:
                # once in the IPv4 part of the tree, switch to IPv4Network
                ipnum = int(naddr)
                mask = network.prefixlen - 128 + 32
                network = IPv4Network((ipnum, mask))
        
        
        subnets = list(network.subnets())
        for bit in (0, 1):
            next_node = self._read_node(node, bit)
            subnet = subnets[bit]

            if next_node > self._metadata.node_count:
                data = self._resolve_data_pointer(next_node)
                yield (subnet, data)
            elif next_node < self._metadata.node_count:
                search_nodes.append((next_node, subnet))



Input the file name

In [ ]:
input_file = 'GeoIP2-City.mmdb'
# input_file = 'GeoIP2-City-Test.mmdb'

output_file = 'out.csv'

In [ ]:
counter=0
write_header = True

row_format = {
  'range': "",
  'continent_code': "",
  'continent': "",
  'country_code': "",
  'country': "",
  'city': "",
  'region': "",
  'region_code': "",
  'latitude': None,
  'longitude': None,
  'location_accuracy_radius': None,
}
out_file = path + output_file

In [ ]:
with maxminddb.open_database(path + input_file) as reader:

  rows = []
  for node in iterable(reader):
    
    row = copy.deepcopy(row_format)
    
    row['range'] = format(node[0])

    d = node[1]

    if 'continent' in d:
      if 'code' in d['continent']:
        row['continent_code'] = d['continent']['code']

      if 'names' in d['continent']:
        if 'en' in d['continent']['names']:
            row['continent'] = d['continent']['names']['en']
    
    if 'registered_country' in d:
      if 'iso_code' in d['registered_country']:
          row['country_code'] = d['registered_country']['iso_code']

      if 'names' in d['registered_country']:
        if 'en' in d['registered_country']['names']:
            row['country'] = d['registered_country']['names']['en']
    
    if 'city' in d:
      if 'names' in d['city']:
          if 'en' in d['city']['names']:
            row['city'] = d['city']['names']['en']

            

    if 'subdivisions' in d:
      if 'names' in d['subdivisions'][0]:
          if 'en' in d['subdivisions'][0]['names']:
            row['region'] = d['subdivisions'][0]['names']['en']
      if 'names' in d['subdivisions'][0]:
            row['region_code'] = d['subdivisions'][0]['iso_code']


    if 'location' in d:
      if 'latitude' in d['location']:
        row['latitude'] = d['location']['latitude']

      if 'longitude' in d['location']:
        row['longitude'] = d['location']['longitude']

      if 'accuracy_radius' in d['location']:
        row['location_accuracy_radius'] = d['location']['accuracy_radius']

    if 'longitude' in d:
      row['longitude'] = d['longitude']
    
    counter += 1
    rows.append(row)

    if counter % 1000 == 0:
      pd.DataFrame(rows).to_csv(out_file, mode='a', header=write_header, index=False)
      write_header = False
      rows = []
      print('.',end='')
    
    if counter % 100000 == 0:
      print('.')
      break


pd.DataFrame(rows).to_csv(out_file, mode='a', header=False, index=False)



Check output

In [ ]:
!head /content/drive/MyDrive/path/out.csv